In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
data = pd.read_excel('data.xlsx')
X = data[['年龄', '有工作', '有自己的房子', '信贷情况']].values
Y = data[['类别']].values

In [3]:
class DTNode:
     
    def __init__(self):
        self.feature = None
        self.value = {}
        self.label = None
        self.entropy = None
        self.subnode = {}

In [4]:
class DecisionTree:
    
    def __init__(self,method,eps):
        self.root = None
        self.method = method
        self.eps = eps
        
    def compute_entropy(self,value):
        value = [v/sum(value) for v in value]
        value = [v for v in value if v!=0]
        value = [-v*math.log(v) for v in value]
        return sum(value)
        
    def fit(self,X,Y):
        self.var_num = X.shape[1]
        df = pd.concat([pd.DataFrame(X,columns = ['var'+str(i) for i in range(self.var_num)]),\
                  pd.DataFrame(Y,columns=['y'])],axis=1)
        self.root = DTNode()
        self.bulid_tree(self.root,df,method=self.method,eps=self.eps)
        
        
    def bulid_tree(self,node,df,method,eps=0):
        node.value = dict(df['y'].value_counts())
        
        ## 均属于一个类
        if len(node.value) == 1:
            node.label = list(node.value.keys())[0]
            node.entropy = 0
        ## 无特征
        elif df.shape[1] == 1:
            node.label = [key for key,value in node.value.items() if value == max(node.value.values())][0]
            node.entropy = self.compute_entropy(list(node.value.values()))
        
        ## 需递归
        else:
            node.label = [key for key,value in node.value.items() if value == max(node.value.values())][0]
            node.entropy = self.compute_entropy(list(node.value.values()))
            ### 计算特征的信息增益
            var_list = list(df.columns[:-1])
            info = {}
            if method == 'ID3':
                for var in var_list:
                    df_tmp = [l[1] for l in list(df[[var,'y']].groupby(var))]
                    cond_info = sum([self.compute_entropy(list(d['y'].value_counts()))*len(d)\
                         for d in df_tmp])/len(df)
                    info[var] = node.entropy-cond_info
            elif method == 'C4.5':
                for var in var_list:
                    df_tmp = [l[1] for l in list(df[[var,'y']].groupby(var))]
                    cond_info = sum([self.compute_entropy(list(d['y'].value_counts()))*len(d)\
                         for d in df_tmp])/len(df)
                    info[var] = (node.entropy-cond_info)/ \
                        self.compute_entropy(list(df[var].value_counts()))
            ### 选出特征
            max_info = max(list(info.values()))
            feature = [key for key in info.keys() if info[key]==max_info][0]
        
            if max_info>=eps:
                node.feature = feature
                
                ### 建立子树
                fea_space = list(np.unique(df[feature].values))
                new_col = [l for l in list(df.columns) if l!=feature]
                for fea in fea_space:
                    node.subnode[fea] = DTNode()
                    self.bulid_tree(node.subnode[fea],df.loc[df[node.feature]==fea,new_col],\
                                method=method,eps=eps)
    
    def print_node(self,node,var_dict,layer=0):
        if len(node.subnode)==0:
            print('|'+'\t|'*layer+'---类别:'+str(node.label)+',分布:'+str(node.value))
        else:
            for key,value in node.subnode.items():
                print('|'+'\t|'*layer+'---'+var_dict[node.feature]+':'+str(key))
                self.print_node(value,var_dict,layer+1)
    
    def print_tree(self,col_name=None):
        var_name = ['var'+str(i) for i in range(self.var_num)]
        if col_name is not None:
            var_dict = dict(zip(var_name,col_name))
        else:
            var_dict = dict(zip(var_name,var_dict))
        
        self.print_node(self.root,var_dict,layer=0)
        
    def predict_sample(self,new_X):
        node = self.root
        while len(node.subnode)!=0:
            try:
                node = node.subnode[new_X[int(node.feature[3])]]
            except:
                break
        return node.label
        
    def predict(self,new_X):
        return np.apply_along_axis(self.predict_sample,axis=1,arr=new_X)
    
    def cut_node(self,node,alpha):
        subnode_entropy = sum([sum(n.value.values())*n.entropy for n in node.subnode.values()])
        if subnode_entropy!=0:
            for subnode in node.subnode.values():
                if subnode.entropy!=0:
                    self.cut_node(subnode,alpha)
        
        subnode_entropy = sum([sum(n.value.values())*n.entropy for n in node.subnode.values()])
        if subnode_entropy!=0:
            delta = sum(node.value.values())*node.entropy-subnode_entropy
            if delta<=alpha*(len(node.subnode)-1):
                node.subnode={}
                node.feature = None
            
    def prun(self,alpha):
        self.cut_node(self.root,alpha)

# Generate

In [5]:
clf = DecisionTree(method='ID3',eps=0)
clf.fit(X,Y)

# Visualize

In [6]:
clf.print_tree(col_name=list(data.columns[:-1]))

|---有自己的房子:否
|	|---有工作:否
|	|	|---类别:0,分布:{0: 6}
|	|---有工作:是
|	|	|---类别:1,分布:{1: 3}
|---有自己的房子:是
|	|---类别:1,分布:{1: 6}


# Prediction

In [7]:
new_X = np.array([['中年', '是', '否', '好'],\
                  ['青年', '否', '否', '非常好']])
clf.predict(new_X)

array([1, 0])

# Pruning

In [8]:
clf.prun(alpha=5)
clf.print_tree(col_name=list(data.columns[:-1]))

|---类别:1,分布:{1: 9, 0: 6}
